# Table of Contents

1. [Importing Libraries](#1.-Importing-Libraries)
2. [Getting the data](#2.-Getting-the-data)
3. [Exploratory Data Analysis](#3.-Exploratory-Data-Analysis) \
    3.1 [Functions](#3.1-Functions) \
    3.2 [EDA](#3.2-EDA) \
    3.3 [Pandas Profiling EDA Report](#3.3-Pandas-Profiling-EDA-Report) \
    3.4 [Choosing 5 commodities](#3.4-Choosing-5-commodities) \
    3.5 [Pandas Profiling - 5 Commodities](#3.5-Pandas-Profiling---5-commodities)

# 1. Importing Libraries

In [1]:
import pandas as pd
from pandas_profiling import ProfileReport

# 2. Getting the data

In [2]:
# commodities_link = 'https://apps.fas.usda.gov/psdonline/downloads/psd_alldata_csv.zip'
commodities_file = 'data/psd_alldata_csv.zip'

all_commodities = pd.read_csv(commodities_file, 
                              compression='zip',
                              # Netherlands Antilles code is NA
                              keep_default_na=False)

all_commodities.head()

,Commodity_Code,Commodity_Description,Country_Code,Country_Name,Market_Year,Calendar_Year,Month,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value
0,577400,"Almonds, Shelled Basis",AF,Afghanistan,2010,2018,10,20,Beginning Stocks,21,(MT),0.0
1,577400,"Almonds, Shelled Basis",AF,Afghanistan,2010,2018,10,125,Domestic Consumption,21,(MT),0.0
2,577400,"Almonds, Shelled Basis",AF,Afghanistan,2010,2018,10,176,Ending Stocks,21,(MT),0.0
3,577400,"Almonds, Shelled Basis",AF,Afghanistan,2010,2018,10,88,Exports,21,(MT),0.0
4,577400,"Almonds, Shelled Basis",AF,Afghanistan,2010,2018,10,57,Imports,21,(MT),0.0


In [3]:
# countries_code_file = 'data/countries_code.xls'

# countries_code_list = pd.read_html(countries_code_file)
# countries_code = pd.DataFrame(countries_code_list[0])

# countries_code

# 3. Exploratory Data Analysis

Understanding the data - columns and values \
What does each row represent?

## 3.1 Functions

In [4]:
def count_unique_values(df: pd.DataFrame):
    
    for column in df.columns:
        
        print(column+': '+str(df[column].nunique()))
        

def filter_by_values(df: pd.DataFrame, column: str, values: list):
    
    return df[df[column].isin(values)].copy()

## 3.2 EDA

In [5]:
all_commodities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1926806 entries, 0 to 1926805
Data columns (total 12 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Commodity_Code         int64  
 1   Commodity_Description  object 
 2   Country_Code           object 
 3   Country_Name           object 
 4   Market_Year            int64  
 5   Calendar_Year          int64  
 6   Month                  int64  
 7   Attribute_ID           int64  
 8   Attribute_Description  object 
 9   Unit_ID                int64  
 10  Unit_Description       object 
 11  Value                  float64
dtypes: float64(1), int64(6), object(5)
memory usage: 176.4+ MB


In [6]:
all_commodities.isna().sum()

Commodity_Code           0
Commodity_Description    0
Country_Code             0
Country_Name             0
Market_Year              0
Calendar_Year            0
Month                    0
Attribute_ID             0
Attribute_Description    0
Unit_ID                  0
Unit_Description         0
Value                    0
dtype: int64

In [7]:
count_unique_values(all_commodities)

Commodity_Code: 63
Commodity_Description: 63
Country_Code: 213
Country_Name: 213
Market_Year: 63
Calendar_Year: 62
Month: 13
Attribute_ID: 71
Attribute_Description: 71
Unit_ID: 11
Unit_Description: 11
Value: 44982


In [9]:
all_commodities[all_commodities.duplicated()]

,Commodity_Code,Commodity_Description,Country_Code,Country_Name,Market_Year,Calendar_Year,Month,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value


Observations to be considered:

- There are 13 different Months

### Months

In [8]:
all_commodities['Month'].unique()

array([10,  8,  0, 11,  3,  2,  7,  4,  1, 12,  6,  5,  9])

In [9]:
commodities_month_0 = all_commodities[all_commodities.Month == 0]

commodities_month_0.head()

,Commodity_Code,Commodity_Description,Country_Code,Country_Name,Market_Year,Calendar_Year,Month,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value
1448,577400,"Almonds, Shelled Basis",GR,Greece,1983,1984,0,20,Beginning Stocks,21,(MT),0.0
1449,577400,"Almonds, Shelled Basis",GR,Greece,1983,1984,0,125,Domestic Consumption,21,(MT),9700.0
1450,577400,"Almonds, Shelled Basis",GR,Greece,1983,1984,0,176,Ending Stocks,21,(MT),345.0
1451,577400,"Almonds, Shelled Basis",GR,Greece,1983,1984,0,88,Exports,21,(MT),4300.0
1452,577400,"Almonds, Shelled Basis",GR,Greece,1983,1984,0,57,Imports,21,(MT),45.0


In [10]:
count_unique_values(commodities_month_0)

Commodity_Code: 25
Commodity_Description: 25
Country_Code: 198
Country_Name: 198
Market_Year: 45
Calendar_Year: 46
Month: 1
Attribute_ID: 50
Attribute_Description: 50
Unit_ID: 6
Unit_Description: 6
Value: 19926


- `Month == 0` can be found in different commodities, countries, years... and so on.

I didn't find anything on the website talking about this.
- When the month is 0, is the observation about the year?

In [11]:
# all_commodities['Month'].replace(0, np.nan, inplace=True)

- Does the dataset have year/0 and year/month records for each attribute?

Here I am trying to understand if `Month == 0` represents the total value for a respective year, that is, if `Value` is the sum of the values in the months for that year when `Month == 0`.

In [52]:
brasil = filter_by_values(df=all_commodities,
                          column='Country_Code',
                          values=['BR'])
                                                    
#brasil['Attribute_Description'].value_counts()

In [13]:
(brasil[(brasil.Commodity_Description == 'Coffee, Green')
         & (brasil.Attribute_Description == 'Production')]
         #& (brasil.Calendar_Year == 2020)]
     .sort_values(['Calendar_Year', 'Month'])
     .tail(25))

,Commodity_Code,Commodity_Description,Country_Code,Country_Name,Market_Year,Calendar_Year,Month,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value
168199,711100,"Coffee, Green",BR,Brazil,1998,1997,0,28,Production,2,(1000 60 KG BAGS),23500.0
168218,711100,"Coffee, Green",BR,Brazil,1999,1998,0,28,Production,2,(1000 60 KG BAGS),35600.0
168237,711100,"Coffee, Green",BR,Brazil,2000,1999,0,28,Production,2,(1000 60 KG BAGS),30800.0
168256,711100,"Coffee, Green",BR,Brazil,2001,2000,0,28,Production,2,(1000 60 KG BAGS),34100.0
168275,711100,"Coffee, Green",BR,Brazil,2002,2001,0,28,Production,2,(1000 60 KG BAGS),35100.0
168332,711100,"Coffee, Green",BR,Brazil,2005,2006,6,28,Production,2,(1000 60 KG BAGS),43600.0
168351,711100,"Coffee, Green",BR,Brazil,2006,2007,6,28,Production,2,(1000 60 KG BAGS),36100.0
168370,711100,"Coffee, Green",BR,Brazil,2007,2008,6,28,Production,2,(1000 60 KG BAGS),46700.0
168389,711100,"Coffee, Green",BR,Brazil,2008,2009,6,28,Production,2,(1000 60 KG BAGS),39100.0
168294,711100,"Coffee, Green",BR,Brazil,2003,2010,6,28,Production,2,(1000 60 KG BAGS),53600.0


When `Month == 0` we only have a single record for that year. So it seems to me that the month's information was lost.

### Attribute Description

https://apps.fas.usda.gov/psdonline/app/index.html#/app/about#G4

- Total Distribution: Exports + domestic consumption + loss + ending stocks.
- Supply = beginning stocks + domestic production + imports.
- Use = domestic consumption + exports + ending stocks.
- Domestic consumption = all possible uses of the commodity: food, feed, seed, waste, and industrial processing.

Stocks include all of the commodity which is not currently in use, regardless of where it is stored: on the farm, in a warehouse or elevator awaiting marketing, at a port awaiting shipment, at a mill awaiting use, or in emergency government reserve.
- Ending stocks = the unused commodity remaining at the end of the marketing year for use in the next year.
- Beginning stocks = the ending stocks carried into the new marketing year from the previous year.

In [14]:
all_commodities['Attribute_Description'].unique()

array(['Beginning Stocks', 'Domestic Consumption', 'Ending Stocks',
       'Exports', 'Imports', 'Production', 'Total Distribution',
       'Total Supply', 'Beef Cows Beg. Stocks', 'Calf Slaughter',
       'Cow Slaughter', 'Cows Entering the Herd',
       'Dairy Cows Beg. Stocks', 'Loss and Residual', 'Total Slaughter',
       'Annual % Change Prod. To Sows', 'Sow Beginning Stocks',
       'Sow Slaughter', 'Commercial Production', 'Non-Comm. Production',
       'Withdrawal From Market', 'Area Harvested',
       'Feed Dom. Consumption', 'FSI Consumption', 'TY Exports',
       'TY Imp. from U.S.', 'TY Imports', 'Yield', 'Arabica Production',
       'Bean Exports', 'Bean Imports', 'Other Production',
       'Roast & Ground Exports', 'Roast & Ground Imports',
       'Robusta Production', 'Rst,Ground Dom. Consum',
       'Soluble Dom. Cons.', 'Soluble Exports', 'Soluble Imports',
       'Loss Dom. Consumption', 'Stocks-to-Use', 'USE Dom. Consumption',
       'Total Use', 'Cows In Milk', 'Co

In [15]:
all_commodities['Commodity_Description'].unique()

array(['Almonds, Shelled Basis', 'Animal Numbers, Cattle',
       'Animal Numbers, Swine', 'Apples, Fresh', 'Barley',
       'Cherries (Sweet&Sour), Fresh', 'Coffee, Green', 'Corn', 'Cotton',
       'Dairy, Butter', 'Dairy, Cheese', 'Dairy, Dry Whole Milk Powder',
       'Dairy, Milk, Fluid', 'Dairy, Milk, Nonfat Dry',
       'Grapefruit, Fresh', 'Grapes, Fresh Table', 'Lemons/Limes, Fresh',
       'Meal, Copra', 'Meal, Cottonseed', 'Meal, Fish',
       'Meal, Palm Kernel', 'Meal, Peanut', 'Meal, Rapeseed',
       'Meal, Soybean', 'Meal, Soybean (Local)', 'Meal, Sunflowerseed',
       'Meat, Beef and Veal', 'Meat, Chicken', 'Meat, Swine', 'Millet',
       'Mixed Grain', 'Oats', 'Oil, Coconut', 'Oil, Cottonseed',
       'Oil, Olive', 'Oil, Palm', 'Oil, Palm Kernel', 'Oil, Peanut',
       'Oil, Rapeseed', 'Oil, Soybean', 'Oil, Soybean (Local)',
       'Oil, Sunflowerseed', 'Oilseed, Copra', 'Oilseed, Cottonseed',
       'Oilseed, Palm Kernel', 'Oilseed, Peanut', 'Oilseed, Rapeseed',
    

In [16]:
(brasil[brasil.Commodity_Description == 'Coffee, Green']
     .groupby(['Commodity_Description', 'Attribute_Description'])
     .sum()
     [['Value']])

Value
Commodity_Description Attribute_Description            
Coffee, Green         Arabica Production      1723550.0
                      Bean Exports            1238536.0
                      Bean Imports                 16.0
                      Beginning Stocks        1292824.0
                      Domestic Consumption     768823.0
                      Ending Stocks           1240815.0
                      Exports                 1374207.0
                      Imports                     571.0
                      Other Production              0.0
                      Production              2090450.0
                      Roast & Ground Exports     1668.0
                      Roast & Ground Imports      555.0
                      Robusta Production       366900.0
                      Rst,Ground Dom. Consum   738527.0
                      Soluble Dom. Cons.        30296.0
                      Soluble Exports          134003.0
                      Soluble Imports               0.0
                      Total Distribution      3383845.0
                      Total Supply            3383845.0

#### Production
all commodities have `Production` attribute?

In [17]:
(all_commodities
     [all_commodities.Attribute_Description == 'Production']
     ['Commodity_Description']
     .nunique())

63

Yes! Above -[here](#3.2-EDA)- we can see that there are 63 different commodities. However, I could notice that not all attributes are present in a commodity.
<br>
<br>


#### Consumption

In [18]:
attributes_series = \
    pd.Series(all_commodities['Attribute_Description'].unique())

consumption_attributes = \
    list(attributes_series[(attributes_series.str.contains('Cons.'))
                           | (attributes_series.str.contains('Use'))])

(all_commodities
     [all_commodities.Attribute_Description.isin(consumption_attributes)]
     [['Commodity_Description', 'Attribute_Description']]
     .value_counts())

Commodity_Description     Attribute_Description 
Sugar, Centrifugal        Human Dom. Consumption    8829
Wheat                     Feed Dom. Consumption     7137
                          Domestic Consumption      7137
                          FSI Consumption           7137
Cotton                    USE Dom. Consumption      7116
                                                    ... 
Oilseed, Soybean (Local)  Food Use Dom. Cons.        116
Oil, Soybean (Local)      Industrial Dom. Cons.      116
                          Food Use Dom. Cons.        116
                          Domestic Consumption       116
Oilseed, Soybean (Local)  Feed Waste Dom. Cons.      116
Length: 167, dtype: int64

In [19]:
(all_commodities
     [all_commodities.Commodity_Description == 'Sugar, Centrifugal']
     ['Attribute_Description']
     .unique())

array(['Beet Sugar Production', 'Beginning Stocks',
       'Cane Sugar Production', 'Ending Stocks', 'Exports',
       'Human Dom. Consumption', 'Imports', 'Other Disappearance',
       'Production', 'Raw Exports', 'Raw Imports',
       'Refined Exp.(Raw Val)', 'Refined Imp.(Raw Val)',
       'Total Disappearance', 'Total Distribution', 'Total Supply'],
      dtype=object)

In [20]:
print('Commodities that do not have "Domestic Consumption" attribute')
(set(all_commodities['Commodity_Description'].unique())
 - set(all_commodities[all_commodities.Attribute_Description == 'Domestic Consumption']
           ['Commodity_Description'].unique()))

Commodities that do not have "Domestic Consumption" attribute


{'Animal Numbers, Cattle',
 'Animal Numbers, Swine',
 'Grapefruit, Fresh',
 'Grapes, Fresh Table',
 'Lemons/Limes, Fresh',
 'Oranges, Fresh',
 'Sugar, Centrifugal',
 'Tangerines/Mandarins, Fresh'}

In [21]:
(all_commodities
     [all_commodities.Commodity_Description == 'Animal Numbers, Cattle']
     ['Attribute_Description']
     .unique())

array(['Beef Cows Beg. Stocks', 'Beginning Stocks', 'Calf Slaughter',
       'Cow Slaughter', 'Cows Entering the Herd',
       'Dairy Cows Beg. Stocks', 'Ending Stocks', 'Exports', 'Imports',
       'Loss and Residual', 'Production', 'Total Distribution',
       'Total Slaughter', 'Total Supply'], dtype=object)

### Unit Description

In [22]:
all_commodities['Unit_Description'].unique()

array(['(MT)', '(1000 HEAD)', '(PERCENT)', '(1000 HA)', '(1000 MT)',
       '(MT/HA)', '(1000 60 KG BAGS)', '1000 480 lb. Bales', '(KG/HA)',
       '(1000 MT CWE)', '(RATIO)'], dtype=object)

## 3.3 Pandas Profiling EDA Report

In [23]:
eda_profile_report = ProfileReport(all_commodities, explorative=True,
                                   title='Commodities - Exploratory Report')

# eda_profile_report

In [24]:
eda_profile_report.to_file('outputs/commodities_exploratory_report.html')

Summarize dataset:   0%|          | 0/25 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## 3.4 Choosing 5 commodities 

In [31]:
coffee = all_commodities[all_commodities.Commodity_Description == 'Coffee, Green']

coffee['Attribute_Description'].unique()

array(['Arabica Production', 'Bean Exports', 'Bean Imports',
       'Beginning Stocks', 'Domestic Consumption', 'Ending Stocks',
       'Exports', 'Imports', 'Other Production', 'Production',
       'Roast & Ground Exports', 'Roast & Ground Imports',
       'Robusta Production', 'Rst,Ground Dom. Consum',
       'Soluble Dom. Cons.', 'Soluble Exports', 'Soluble Imports',
       'Total Distribution', 'Total Supply'], dtype=object)

In [39]:
coffee[(coffee.Market_Year == 2003) 
       & (coffee.Country_Name == 'Algeria')]

,Commodity_Code,Commodity_Description,Country_Code,Country_Name,Market_Year,Calendar_Year,Month,Attribute_ID,Attribute_Description,Unit_ID,Unit_Description,Value
163022,711100,"Coffee, Green",AG,Algeria,2003,2012,6,29,Arabica Production,2,(1000 60 KG BAGS),0.0
163023,711100,"Coffee, Green",AG,Algeria,2003,2012,6,90,Bean Exports,2,(1000 60 KG BAGS),0.0
163024,711100,"Coffee, Green",AG,Algeria,2003,2012,6,58,Bean Imports,2,(1000 60 KG BAGS),1800.0
163025,711100,"Coffee, Green",AG,Algeria,2003,2012,6,20,Beginning Stocks,2,(1000 60 KG BAGS),0.0
163026,711100,"Coffee, Green",AG,Algeria,2003,2012,6,125,Domestic Consumption,2,(1000 60 KG BAGS),1805.0
163027,711100,"Coffee, Green",AG,Algeria,2003,2012,6,176,Ending Stocks,2,(1000 60 KG BAGS),0.0
163028,711100,"Coffee, Green",AG,Algeria,2003,2012,6,88,Exports,2,(1000 60 KG BAGS),0.0
163029,711100,"Coffee, Green",AG,Algeria,2003,2012,6,57,Imports,2,(1000 60 KG BAGS),1805.0
163030,711100,"Coffee, Green",AG,Algeria,2003,2012,6,56,Other Production,2,(1000 60 KG BAGS),0.0
163031,711100,"Coffee, Green",AG,Algeria,2003,2012,6,28,Production,2,(1000 60 KG BAGS),0.0


Information on Coffee: \
Market Year: 2003 \
Country: Algeria \

- Calendar Year: 2012
- Imports: 1805
    - Bean Imports: 1800
    - Soluble Imports: 5
- Domestic Consumption: 1805
    - Rst,Ground Dom. Consum: 1800
    - Soluble Dom. Cons.: 5
    
    
Besides coffee, let's choose more commodities to work on.

In [44]:
oranges = filter_by_values(df=all_commodities,
                           column='Commodity_Description',
                           values=['Oranges, Fresh'])

oranges['Attribute_Description'].unique()

array(['Exports', 'For Processing', 'Fresh Dom. Consumption', 'Imports',
       'Production', 'Total Distribution', 'Total Supply'], dtype=object)

In [51]:
lemons_limes = filter_by_values(df=all_commodities,
                                column='Commodity_Description',
                                values=['Lemons/Limes, Fresh'])

lemons_limes['Attribute_Description'].unique()

array(['Exports', 'For Processing', 'Fresh Dom. Consumption', 'Imports',
       'Production', 'Total Distribution', 'Total Supply'], dtype=object)

In [59]:
commodities_description = all_commodities['Commodity_Description'].unique()
attributes_wanted = list(oranges['Attribute_Description'].unique())
commodities_same_attributes = []

for commodity in commodities_description:
    
    commodity_df = filter_by_values(df=all_commodities,
                                    column='Commodity_Description',
                                    values=[commodity])
    
    if list(commodity_df['Attribute_Description'].unique()) == attributes_wanted:
        commodities_same_attributes.append(commodity)
    
commodities_same_attributes

['Grapefruit, Fresh',
 'Lemons/Limes, Fresh',
 'Oranges, Fresh',
 'Tangerines/Mandarins, Fresh']

## 3.5 Pandas Profiling - 5 commodities

In [68]:
commodities_description_explore = commodities_same_attributes + ['Coffee, Green']

commodities = filter_by_values(df=all_commodities,
                               column='Commodity_Description',
                               values=commodities_description_explore)

In [69]:
eda_profile_report_5_commodities = \
    ProfileReport(commodities, explorative=True,
                  title='5 Commodities - Exploratory Report')

In [71]:
eda_profile_report_5_commodities.to_file('outputs/5_commodities_exploratory_report.html')

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]